In [2]:
"""
Idea: recuperar los headlines del espectador, el tiempo, 2orillas, silla vacia, publimetro usando bs4

Al inicio, obtener info solo de la pág. principal. Luego, buscar dentro de cada artículo (tags) (eg. la sila vacia no tiene tags en index).

Luego: clustering? comparar con otros periódicos?
"""
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [3]:
## ESPECTADOR

page = requests.get("https://www.elespectador.com/noticias/").content # página principal del espectador
soup = BeautifulSoup(page, features="html.parser")
articulos = soup.findAll("div", class_= "node--article")
base_url="https://www.elespectador.com"
print(f"Se encontraron {len(articulos)} artículos.")

titulos=[]
categorias=[]
links=[]

for articulo in articulos:
    categorias.append(articulo.find("div",class_="node-term").a.string)
    titulos.append(articulo.find("div",class_="node-title").a.string)
    link=base_url+articulo.find("div",class_="node-title").a["href"]
    links.append(link)
    #TODO obtener tags a partir del link

espectador=pd.DataFrame(data={"titulo":titulos,"categoria":categorias,"link":links})
espectador.head()


Se encontraron 41 artículos.


,titulo,categoria,link
0,Presidente Duque declara la emergencia sanitar...,Política,https://www.elespectador.com/coronavirus/presi...
1,Dólar en Colombia supera los $4.000,Economía,https://www.elespectador.com/coronavirus/dolar...
2,China da por superado el pico de transmisiones...,El Mundo,https://www.elespectador.com/noticias/el-mundo...
3,Discurso de Trump no logra tranquilizar a merc...,Economía,https://www.elespectador.com/coronavirus/discu...
4,Pilotos de fracking: ¿el gobierno se autohabil...,Judicial,https://www.elespectador.com/noticias/judicial...


In [4]:
## EL TIEMPO

page = requests.get("https://www.eltiempo.com/").content # página principal del espectador
soup = BeautifulSoup(page, features="html.parser")
base_url="https://www.eltiempo.com" #para los href

articulos=soup.findAll("article")
print(f"Se encontraron {len(articulos)} artículos.")

titulos=[]
categorias=[]
autores=[]
links=[]

#TODO REMOVER DUPLICADOS Y COSAS INÚTILES (indidcadores económicos, galería, video)
for articulo in articulos:
        link=None

        titulo=articulo.find(class_=re.compile("^(title\-container)|(col\-title)", re.IGNORECASE))
        if titulo is not None:
            #print(titulo)  
            link=base_url+titulo.a["href"]
            titulo=titulo.a.contents[0]
        
        autor=articulo.find(itemprop="author")
        if autor is not None:
            autor=autor.find(itemprop="name").string

       
        categoria=articulo.find(class_=re.compile("^(category-published)|(supratitle)", re.IGNORECASE))
        #print(categoria) 
        if categoria is not None:
            if categoria.a is not None:              
                categoria=categoria.a.contents[0]
            else:
                 categoria=None  

        links.append(link)
        titulos.append(titulo)
        categorias.append(categoria)
        autores.append(autor)

eltiempo=pd.DataFrame(data={"titulo":titulos,"categoria":categorias,"autor":autores,"link":links})
eltiempo.head()

Se encontraron 98 artículos.


,titulo,categoria,autor,link
0,"Tom Hanks, Estéreo Picnic, NBA, Feria del Libr...",None,None,https://www.eltiempo.com/podcast/el-primer-caf...
1,Se declara emergencia sanitaria nacional por c...,coronavirus,Luisa Mercado,https://www.eltiempo.com/politica/gobierno/cor...
2,Dólar abre por encima de los 4.000 pesos batie...,None,Martha Morales,https://www.eltiempo.com/economia/sectores/dol...
3,"Según la OMS, la pandemia del coronavirus toda...",None,Camilo Hernández,https://www.eltiempo.com/salud/coronavirus-oms...
4,Alertan posible riesgo de coronavirus en cárce...,None,María Fernanda Arbeláez,https://www.eltiempo.com/justicia/servicios/al...


In [5]:
## LAS DOS ORILLAS

page = requests.get("https://www.las2orillas.co/").content # página principal del espectador
soup = BeautifulSoup(page, features="html.parser")

#tipos: artículo normal, carrusel grande, carrusel video, carrusel normal
articulos=soup.findAll("div",class_="post")
carruseles=soup.findAll("div",class_="carousel-inner") 
articulos_carrusel=[]
for carrusel in carruseles:
    articulos_carrusel.extend(carrusel.findAll("div",class_="item"))    

print(f"Se encontraron {len(articulos)+len(articulos_carrusel)} artículos.")

titulos=[]
links=[]
descripciones=[]

for articulo in articulos:
    titulo=articulo.find(class_="entry-title").a.string
    link=articulo.find(class_="entry-title").a["href"]
    desc=articulo.find(class_="entry-content").string.strip()

    titulos.append(titulo)
    links.append(link)
    descripciones.append(desc)

for articulo in articulos_carrusel:

    # esto realmente no lo llenan
    # cat=articulo.find("a",class_="image-top")
    # if cat is not None:
    #     cat=cat.string

    titulo=articulo.find(class_="entry-title")
    if titulo.a is not None:
        titulo=titulo.a.string
    else:
        titulo=titulo.string

    link=articulo.find("a",title=re.compile("^.*"))["href"]
    desc=articulo.find("div",class_="text-container")
    if desc is not None:
        desc=str(desc.p.string).strip()

    titulos.append(titulo)
    descripciones.append(desc)
    links.append(link)

dosorillas=pd.DataFrame(data={"titulo":titulos,"descripcion":descripciones,"link":links})
dosorillas.head()

    


Se encontraron 25 artículos.


,titulo,descripcion,link
0,El asesinato del campesino de Piendamó que pue...,"El caso de Ulpiano Ortiz, otro de los más de 5...",https://www.las2orillas.co/el-asesinato-del-ca...
1,"Tom Hanks, la primera estrella de Hollywood en...",Su esposa Rita Wilson también tiene el virus. ...,https://www.las2orillas.co/tom-hanks-la-primer...
2,"Tocilizumab, el medicamento contra la artritis...",El oncólogo italiano Paulo Ascierto de Pascale...,https://www.las2orillas.co/tocilizumab-el-medi...
3,Hassam reconoce que su cáncer no tiene cura,"El pesar de sus seguidores es absoluto cuando,...",https://www.las2orillas.co/nada-que-hacerhassa...
4,La promoción de la asamblea del BID 2020 que h...,Transmetro lanzó la tarjeta conmemorativa de l...,https://www.las2orillas.co/la-promocion-de-la-...


In [6]:
# PUBLÍMETRO
page = requests.get("https://www.publimetro.co/co/").content # página principal del espectador
soup = BeautifulSoup(page, features="html.parser")

#tipos: grande (con related), normal, mini
articulos=soup.findAll("div",class_="article-view")
print(f"Se encontraron {len(articulos)} artículos.")

#estilo de vida y opinión es diferente

titulos=[]
links=[]
categorias=[]

for articulo in articulos:

    titulo=articulo.find(class_="tit").a.string
    link=articulo.find(class_="tit").a["href"]
    cat=articulo.find(class_="article-category-list")
    if cat is not None:
        cat=cat.find("a").string if cat.find("a") is not None else None

    titulos.append(titulo)
    links.append(link)
    categorias.append(cat)

publimetro=pd.DataFrame(data={"titulo":titulos,"categoria":categorias,"link":links})
publimetro.head()

Se encontraron 63 artículos.


,titulo,categoria,link
0,Iván Duque declara emergencia sanitaria y orde...,Noticias,https://www.publimetro.co/co/noticias/2020/03/...
1,"James Rodríguez, en cuarentena con todo el Rea...",Noticias,https://www.publimetro.co/co/noticias/2020/03/...
2,Actor italiano quedó atrapado junto al cadáver...,None,https://www.publimetro.co/co/mundo/2020/03/12/...
3,Federación Colombiana de Tenis tomó drástica m...,None,https://www.publimetro.co/co/deportes/2020/03/...
4,"""Crecerá el número de casos y es posible que t...",None,https://www.publimetro.co/co/noticias/2020/03/...


In [68]:
# SEMANA

page = requests.get("https://www.semana.com/").content # página principal del espectador
soup = BeautifulSoup(page, features="html.parser")
base_url="https://www.semana.com"

articulos=soup.findAll("article")
print(f"Se encontraron {len(articulos)} artículos.")


titulos=[]
categorias=[]
links=[]
descripciones=[]

for articulo in articulos:

    titulo=articulo.find("a",class_=re.compile("^article.*link")).string
    if titulo is not None:
        titulo=titulo.strip()
    else:
        titulo=re.sub("\s{2,}"," ",articulo.find("a",class_=re.compile("^article.*link")).text).strip()    

    link=base_url+articulo.find(class_=re.compile("^article.*link"))["href"]

    desc=articulo.find("p").string
    desc=desc.strip() if desc is not None else None

    cat=articulo.find(class_=re.compile("tag"))
    if cat is not None:
        cat=cat.string.strip()

    titulos.append(titulo)
    categorias.append(cat)
    links.append(link)
    descripciones.append(desc)

semana=pd.DataFrame(data={"titulo":titulos,"categoria":categorias,"link":links,"descripcion":descripciones})
semana.describe()

Se encontraron 60 artículos.


,titulo,categoria,link,descripcion
count,60,59,60,59
unique,60,27,60,59
top,Sentencian a Harvey Weinstein a 23 años de pri...,MUNDO,https://www.semana.com/deportes/articulo/conme...,"Así lo decidió la Corte Constitucional, que ta..."
freq,1,11,1,1
